## Packages download 

In [ ]:
!pip install open3d scipy spektral

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 408.6 MB 27 kB/s 
     |████████████████████████████████| 140 kB 54.9 MB/s 
     |████████████████████████████████| 8.8 MB 34.0 MB/s 
     |████████████████████████████████| 1.1 MB 40.7 MB/s 
     |████████████████████████████████| 3.1 MB 39.0 MB/s 
     |████████████████████████████████| 423 kB 52.7 MB/s 
     |████████████████████████████████| 344 kB 54.5 MB/s 
     |████████████████████████████████| 54 kB 2.2 MB/s 
     |████████████████████████████████| 9.7 MB 49.5 MB/s 
     |████████████████████████████████| 1.6 MB 39.3 MB/s 
     |████████████████████████████████| 1.2 MB 40.7 MB/s 
     |████████████████████████████████| 54 kB 2.9 MB/s 
     |████████████████████████████████| 59 kB 5.6 MB/s 
     |████████████████████████████████| 271 kB 41.2 MB/s 
     |████████████████████████████████| 80 kB 7.2 MB/s 
     |████████████████████████████████|

## Libraries and Loading the point clouds :

In [ ]:
import open3d as o3d
import tensorflow as tf
import numpy as np
import spektral
from tensorflow.keras.layers import Dropout, Input
from tensorflow import keras
from tensorflow.keras import layers
from scipy import sparse
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Dropout
from spektral.layers import GCNConv, GlobalSumPool
from spektral.data import BatchLoader
from spektral.data.loaders import Loader
from spektral.layers import ChebConv
from spektral.data import Dataset
from scipy.spatial import distance_matrix
from spektral.data import Graph
import os
import glob
from scipy.sparse import save_npz
from spektral.layers import GATConv
from spektral.data.loaders import SingleLoader


## Transforming the graph Signal from RGB to YUV :

In [ ]:
def rgb_to_yuv(rgb):
    y= 0.299 * rgb[0] + 0.587*rgb[1]+ 0.114 * rgb[2]
    u = 0.492 * (rgb[2]-y)
    v=0.877* (rgb[0]-y)
    return [y, u ,v]
def RGB2YUV( rgb ):
     
    m = np.array([[ 0.29900, -0.16874,  0.50000],
                 [0.58700, -0.33126, -0.41869],
                 [ 0.11400, 0.50000, -0.08131]])
     
    yuv = np.dot(rgb,m)
    yuv[:,:,1:]+=128.0
    return yuv

In [ ]:
def yuv_to_rgb(yuv):
  Y,U,V = yuv
  R = Y + 1.140 * V
  G = Y - 0.395 * U - 0.581 * V
  B = Y + 2.032 * U
  # R = R.astype(np.float) / 255.0
  # G = G.astype(np.float) / 255.0
  # B= B.astype(np.float) / 255.0
  return [R, G ,B]
def YUV2RGB( yuv ):
      
    m = np.array([[ 1.0, 1.0, 1.0],
                 [-0.000007154783816076815, -0.3441331386566162, 1.7720025777816772],
                 [ 1.4019975662231445, -0.7141380310058594 , 0.00001542569043522235] ])
    
    rgb = np.dot(yuv,m)
    rgb[:,:,0]-=179.45477266423404
    rgb[:,:,1]+=135.45870971679688
    rgb[:,:,2]-=226.8183044444304
    return rgb

## Importing Data :  

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive') 

Mounted at /content/gdrive


In [ ]:
!unzip  "/content/model_Predlift_r03_U_ckpt(0-100).zip" -d "/content/u_model"
!unzip  "/content/model_Predlift_r03_V_ckpt(0-100).zip" -d "/content/v_model"
!unzip  "/content/model_Predlift_r03_Y_ckpt3(0-100) (1).zip" -d "/content/y_model"

Archive:  /content/model_Predlift_r03_U_ckpt(0-100).zip
   creating: /content/u_model/content/model_Predlift_r03_U_ckpt(0-100)/
   creating: /content/u_model/content/model_Predlift_r03_U_ckpt(0-100)/assets/
   creating: /content/u_model/content/model_Predlift_r03_U_ckpt(0-100)/variables/
  inflating: /content/u_model/content/model_Predlift_r03_U_ckpt(0-100)/variables/variables.data-00000-of-00001  
  inflating: /content/u_model/content/model_Predlift_r03_U_ckpt(0-100)/variables/variables.index  
  inflating: /content/u_model/content/model_Predlift_r03_U_ckpt(0-100)/keras_metadata.pb  
  inflating: /content/u_model/content/model_Predlift_r03_U_ckpt(0-100)/saved_model.pb  
Archive:  /content/model_Predlift_r03_V_ckpt(0-100).zip
   creating: /content/v_model/content/model_Predlift_r03_V_ckpt(0-100)/
   creating: /content/v_model/content/model_Predlift_r03_V_ckpt(0-100)/assets/
   creating: /content/v_model/content/model_Predlift_r03_V_ckpt(0-100)/variables/
  inflating: /content/v_model/c

In [ ]:
!unrar x -Y "/content/r01.rar" "/content/"


UNRAR 5.50 freeware      Copyright (c) 1993-2017 Alexander Roshal


Extracting from /content/r01.rar

Extracting  /content/original.ply                                         66%  OK 
Extracting  /content/decoded.ply                                          99%  OK 
All OK


## Loss function :

In [ ]:
def custom_loss_function(y_true, y_pred):
  squared_difference = tf.square(y_true - y_pred)
  return tf.reduce_mean(squared_difference, axis=-1)

In [ ]:
def corrector(s):
    for a in range(0,len(s)):
        if (s[a]=='\''):
            s[a]='/'
    return s

In [ ]:
!rm -r patches

rm: cannot remove 'patches': No such file or directory


## Patchs creation : 


In [ ]:
!rm -r /content/spektral/datasets/PatchDatasets
!mkdir spektral spektral/datasets
!mkdir patches
dataset=[]

rm: cannot remove '/content/spektral/datasets/PatchDatasets': No such file or directory


In [ ]:
class PatchDatasets(Dataset):
    """
    A dataset for the graphs
    """
    def __init__(self, patch_n=None, *args, **kwargs):
        self.patch_n = patch_n
        super(PatchDatasets, self).__init__(*args, **kwargs)

    def read(self):
      output = []
      if self.patch_n == None :
        return output
      data = np.load(self.patch_n,allow_pickle=True)
      output.append(
            Graph(x=data['x'], a=data['a'])
        )
      return output
    def download(self):
        # Create the directory
        if os.path.exists("/content/spektral/datasets/PatchDatasets/"):
          return 
        os.mkdir('/content/spektral/datasets/PatchDatasets/')
        for name in glob.glob('/content/decoded.ply'):

            pcd = o3d.io.read_point_cloud(corrector(name))
            i=0
            # Patch creation
            while(i<=int(len(pcd.points)/2048)):
                nd = []
                rgb= []
                if(i==int(len(pcd.points)/2048)):
                    nd = pcd.points[i*2048:]
                    rgb= pcd.colors[i*2048:]
                    while(len(nd)!=2048):
                        nd.append(nd[-1])
                        rgb.append(rgb[-1])
                else:
                    nd = pcd.points[i*2048:(i+1)*2048]
                    rgb = pcd.colors[i*2048:(i+1)*2048]
                i+=1
                # RGB to YUV : 
                yuv_colors=[]
                yuv_y_colors=[]
                for clr in range(0,len(rgb)):
                    yuv_colors.append(rgb_to_yuv(rgb[clr]))
                # Graph signals
                xY = np.array(yuv_colors)[:,0]
                xU = np.array(yuv_colors)[:,1]
                xV = np.array(yuv_colors)[:,2]

                temp_pcd = o3d.geometry.PointCloud()
                temp_pcd.points = nd
                # Adjacency Matrix
                a =  distance_matrix(nd , nd)
                a =   np.exp(-1*a)
                # a= sparse.csr_matrix(A)
                # Saving patches to files
                path = "/content/spektral/datasets/PatchDatasets"
                data_name= name.split('/')[-2]
                filenameY = os.path.join(path, f'graph_Y_{data_name}_{i}')
                filenameU = os.path.join(path, f'graph_U_{data_name}_{i}')
                filenameV = os.path.join(path, f'graph_V_{data_name}_{i}')
                np.savez(filenameY, x=xY, a=a)
                np.savez(filenameU, x=xU, a=a)
                np.savez(filenameV, x=xV, a=a)
                o3d.io.write_point_cloud(f"/content/patches/patch_{i}.ply",temp_pcd)


dataset = PatchDatasets()

[Open3D WARNING] Read PLY failed: unable to read file: /content/decoded.ply


In [ ]:
def custom_loss_function(y_true, y_pred):
  squared_difference = tf.square(y_true - y_pred)
  return tf.reduce_mean(squared_difference, axis=-1)

class Predlift_Y_r1(Model):

    def __init__(self, n_channels, n_labels):
      super().__init__()
      self.graph_conv = ChebConv(channels=n_channels)
      self.graph_conv2 = ChebConv(channels=n_channels)
      self.gat_conv = GATConv(64)
      self.pool = GlobalSumPool()
      self.dense = Dense(n_labels, 'relu')

    def call(self, inputs):
      if len(inputs) == 2:
        x, a = inputs
      # Branch structure 
      out1 = self.graph_conv([x, a])
      out2 = self.gat_conv([out1, a])
      out3 = self.graph_conv2([out2, a])
      out4 = self.gat_conv([out3, a])
      out5 = self.graph_conv2([out4, a])
      out = tf.keras.layers.Concatenate(axis=1)([out1,out2,out3,out4,out5])
      # Dense Layers 
      out = self.pool(out)
      out = self.dense(out)
      # Residual learning
      x_res = x[:,:,0]
      out = tf.keras.layers.Add()([out, x_res])
      return out
class Predlift_U_r1(Model):

    def __init__(self, n_channels, n_labels):
      super().__init__()
      self.graph_conv = ChebConv(channels=n_channels)
      self.graph_conv2 = ChebConv(channels=n_channels)
      self.gat_conv = GATConv(64)
      self.pool = GlobalSumPool()
      self.dense = Dense(n_labels, 'relu')

    def call(self, inputs):
      if len(inputs) == 2:
        x, a = inputs
      # Branch structure 
      out1 = self.graph_conv([x, a])
      out2 = self.gat_conv([out1, a])
      out3 = self.graph_conv2([out2, a])
      out4 = self.gat_conv([out3, a])
      out5 = self.graph_conv2([out4, a])
      out = tf.keras.layers.Concatenate(axis=1)([out1,out2,out3,out4,out5])
      # Dense Layers 
      out = self.pool(out)
      out = self.dense(out)
      # Residual learning
      x_res = x[:,:,0]
      out = tf.keras.layers.Add()([out, x_res])
      return out
class Predlift_V_r1(Model):

    def __init__(self, n_channels, n_labels):
      super().__init__()
      self.graph_conv = ChebConv(channels=n_channels)
      self.graph_conv2 = ChebConv(channels=n_channels)
      self.gat_conv = GATConv(64)
      self.pool = GlobalSumPool()
      self.dense = Dense(n_labels, 'relu')

    def call(self, inputs):
      if len(inputs) == 2:
        x, a = inputs
      # Branch structure 
      out1 = self.graph_conv([x, a])
      out2 = self.gat_conv([out1, a])
      out3 = self.graph_conv2([out2, a])
      out4 = self.gat_conv([out3, a])
      out5 = self.graph_conv2([out4, a])
      out = tf.keras.layers.Concatenate(axis=1)([out1,out2,out3,out4,out5])
      # Dense Layers 
      out = self.pool(out)
      out = self.dense(out)
      # Residual learning
      x_res = x[:,:,0]
      out = tf.keras.layers.Add()([out, x_res])
      return out

In [ ]:
loaded_y = keras.models.load_model(
    '/content/y_model/content/model_Predlift_r03_Y_ckpt4(0-200)', custom_objects={"Predlift_U_r1": Predlift_Y_r1 ,"custom_loss_function":custom_loss_function}
)

In [ ]:
loaded_u = keras.models.load_model(
    '/content/u_model/content/model_Predlift_r03_U_ckpt(0-100)', custom_objects={"Predlift_U_r1": Predlift_U_r1 ,"custom_loss_function":custom_loss_function}
)

In [ ]:
loaded_v = keras.models.load_model(
    '/content/v_model/content/model_Predlift_r03_V_ckpt(0-100)', custom_objects={"Predlift_V_r1": Predlift_V_r1 ,"custom_loss_function":custom_loss_function}
)

In [ ]:
!mkdir cleaned_data

In [ ]:
for name in glob.glob('/content/patches/*'):
  
  patch_number = name.split('.')[0].split('_')[-1]

  y_graph =  PatchDatasets(f"/content/spektral/datasets/PatchDatasets/graph_Y_content_{patch_number}.npz")
  u_graph =  PatchDatasets(f"/content/spektral/datasets/PatchDatasets/graph_U_content_{patch_number}.npz")
  v_graph =  PatchDatasets(f"/content/spektral/datasets/PatchDatasets/graph_V_content_{patch_number}.npz")
  if len(y_graph[0].x)==len(u_graph[0].x)==len(v_graph[0].x)==2048 :
    y_predictions = loaded_y.predict([tf.reshape(y_graph[0].x,[1,2048,1]),  tf.reshape(y_graph[0].a,[1,2048,2048])])
    u_predictions = loaded_u.predict([tf.reshape(u_graph[0].x,[1,2048,1]),  tf.reshape(u_graph[0].a,[1,2048,2048])])
    v_predictions = loaded_v.predict([tf.reshape(v_graph[0].x,[1,2048,1]),  tf.reshape(v_graph[0].a,[1,2048,2048])])
  
    temp_pcd = o3d.geometry.PointCloud()

    pcd = o3d.io.read_point_cloud(f'/content/patches/patch_{patch_number}.ply')
    temp_pcd.points = pcd.points

    yuv_colors =  np.array([y_predictions[0],u_predictions[0],v_predictions[0]])
    yuv_colors= np.transpose(yuv_colors)
    rgb_colors=[]
    for clr in range(0,len(yuv_colors)):
      rgb_colors.append(yuv_to_rgb(yuv_colors[clr]))
    temp_pcd.colors=o3d.utility.Vector3dVector(rgb_colors)
    o3d.io.write_point_cloud(f"/content/patches/patch_{patch_number}.ply",temp_pcd)


/usr/local/lib/python3.7/dist-packages/spektral/data/graph.py:67: UserWarning: x was automatically reshaped to (2048, 1)
  warnings.warn(f"x was automatically reshaped to {x.shape}")


[Open3D WARNING] Write Ply clamped color value to valid range
[Open3D WARNING] Write Ply clamped color value to valid range
[Open3D WARNING] Write Ply clamped color value to valid range
[Open3D WARNING] Write Ply clamped color value to valid range
[Open3D WARNING] Write Ply clamped color value to valid range
[Open3D WARNING] Write Ply clamped color value to valid range
[Open3D WARNING] Write Ply clamped color value to valid range
[Open3D WARNING] Write Ply clamped color value to valid range
[Open3D WARNING] Write Ply clamped color value to valid range
[Open3D WARNING] Write Ply clamped color value to valid range
[Open3D WARNING] Write Ply clamped color value to valid range
[Open3D WARNING] Write Ply clamped color value to valid range
[Open3D WARNING] Write Ply clamped color value to valid range
[Open3D WARNING] Write Ply clamped color value to valid range
[Open3D WARNING] Write Ply clamped color value to valid range
[Open3D WARNING] Write Ply clamped color value to valid range
[Open3D 

In [ ]:
!zip -r "/content/cleaned.zip" "/content/patches"

  adding: content/patches/ (stored 0%)
  adding: content/patches/patch_259.ply (deflated 81%)
  adding: content/patches/patch_403.ply (deflated 88%)
  adding: content/patches/patch_84.ply (deflated 89%)
  adding: content/patches/patch_357.ply (deflated 88%)
  adding: content/patches/patch_4.ply (deflated 88%)
  adding: content/patches/patch_92.ply (deflated 88%)
  adding: content/patches/patch_133.ply (deflated 88%)
  adding: content/patches/patch_158.ply (deflated 89%)
  adding: content/patches/patch_440.ply (deflated 86%)
  adding: content/patches/patch_105.ply (deflated 87%)
  adding: content/patches/patch_42.ply (deflated 89%)
  adding: content/patches/patch_384.ply (deflated 89%)
  adding: content/patches/patch_425.ply (deflated 87%)
  adding: content/patches/patch_453.ply (deflated 89%)
  adding: content/patches/patch_50.ply (deflated 88%)
  adding: content/patches/patch_58.ply (deflated 88%)
  adding: content/patches/patch_363.ply (deflated 88%)
  adding: content/patches/patch_3

## Assembling Patches : 

In [ ]:
!unzip "/content/cleaned.zip" -d "/content/patches/"

Archive:  /content/cleaned.zip
   creating: /content/patches/content/patches/
  inflating: /content/patches/content/patches/patch_259.ply  
  inflating: /content/patches/content/patches/patch_403.ply  
  inflating: /content/patches/content/patches/patch_84.ply  
  inflating: /content/patches/content/patches/patch_357.ply  
  inflating: /content/patches/content/patches/patch_4.ply  
  inflating: /content/patches/content/patches/patch_92.ply  
  inflating: /content/patches/content/patches/patch_133.ply  
  inflating: /content/patches/content/patches/patch_158.ply  
  inflating: /content/patches/content/patches/patch_440.ply  
  inflating: /content/patches/content/patches/patch_105.ply  
  inflating: /content/patches/content/patches/patch_42.ply  
  inflating: /content/patches/content/patches/patch_384.ply  
  inflating: /content/patches/content/patches/patch_425.ply  
  inflating: /content/patches/content/patches/patch_453.ply  
  inflating: /content/patches/content/patches/patch_50.ply 

In [ ]:
all_points = []
all_colors = []
for name in glob.glob('/content/patches/content/patches/*'):
  pcd = o3d.io.read_point_cloud(name)
  print(pcd)
  for i in range(0,len(pcd.points)):
    all_points.append(pcd.points[i])
  for i in range(0,len(pcd.colors)):
    all_colors.append(pcd.colors[i])


PointCloud with 2048 points.
PointCloud with 2048 points.
PointCloud with 2048 points.
PointCloud with 2048 points.
PointCloud with 2048 points.
PointCloud with 2048 points.
PointCloud with 2048 points.
PointCloud with 2048 points.
PointCloud with 2048 points.
PointCloud with 2048 points.
PointCloud with 2048 points.
PointCloud with 2048 points.
PointCloud with 2048 points.
PointCloud with 2048 points.
PointCloud with 2048 points.
PointCloud with 2048 points.
PointCloud with 2048 points.
PointCloud with 2048 points.
PointCloud with 2048 points.
PointCloud with 2048 points.
PointCloud with 2048 points.
PointCloud with 2048 points.
PointCloud with 2048 points.
PointCloud with 2048 points.
PointCloud with 2048 points.
PointCloud with 2048 points.
PointCloud with 2048 points.
PointCloud with 2048 points.
PointCloud with 2048 points.
PointCloud with 2048 points.
PointCloud with 2048 points.
PointCloud with 2048 points.
PointCloud with 2048 points.
PointCloud with 2048 points.
PointCloud wit

In [ ]:
while(len(all_points)!=len(all_colors)):
  all_colors.append(all_colors[-1])

In [ ]:
len(all_colors) 

962560

In [ ]:
pcd = o3d.geometry.PointCloud()
pcd.points= o3d.utility.Vector3dVector(all_points[:-1084])
pcd.colors = o3d.utility.Vector3dVector(all_colors[:-1084])

In [ ]:
pcd.colors[-5]

array([0.02352941, 0.06666667, 0.03921569])

In [ ]:
o3d.io.write_point_cloud('cleaned.ply',pcd)

True

In [ ]:
pcd2 = o3d.io.read_point_cloud("/content/cleaned.ply")
pcd1 = o3d.io.read_point_cloud("/content/original.ply")

In [ ]:
pcd2

PointCloud with 961476 points.